In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install dependencies
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
video = 'highway_morning.MOV'

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
%cd /content/yolov5/
!python detect.py --weights '/content/gdrive/My Drive/WongManKiat_1161204289/weights/last.pt' --source '/content/gdrive/My Drive/WongManKiat_1161204289/original_video/$video' --save-txt

/content/yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=True, source='/content/gdrive/My Drive/WongManKiat_1161204289/original_video/highway_morning.MOV', update=False, view_img=False, weights=['/content/gdrive/My Drive/WongManKiat_1161204289/weights/last.pt'])
YOLOv5 v4.0-80-gf8464b4 torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 232 layers, 7273488 parameters, 0 gradients, 16.9 GFLOPS
video 1/1 (1/233) /content/gdrive/My Drive/WongManKiat_1161204289/original_video/highway_morning.MOV: 384x640 2 cars, Done. (0.030s)
video 1/1 (2/233) /content/gdrive/My Drive/WongManKiat_1161204289/original_video/highway_morning.MOV: 384x640 1 car, Done. (0.012s)
video 1/1 (3/233) /content/gdrive/My Drive/WongManKiat_1161204289/original_video/highway_morning.MOV: 384x640 Done. (0.012s)
video 1/1 (4/233) /content/gdri

In [ ]:
import os
exp = sorted(os.listdir('/content/yolov5/runs/detect'))[-1]

In [ ]:
# !zip -r /content/file.zip /content/yolov5/runs/detect/exp5/labels

In [ ]:
%cp /content/yolov5/runs/detect/$exp/$video /content/gdrive/My\ Drive/WongManKiat_1161204289/object_detection_video

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import pickle
import time

def abs_sobel_thresh(img, sobel_kernel=3, orient='x', thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

def mag_threshold(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output    

def binary_thresh(channel, thresh = (200, 255)):
        binary = np.zeros_like(channel)
        binary[(channel > thresh[0]) & (channel <= thresh[1])] = 1

        return binary

def combined_threshold(img, kernel=3):
    if(np.sum(img)>378572516):
        grad_thresh=(50,255)
        mag_thresh=(50,255)
        l_thresh=(180,255)
        g_thresh=(180,255)
        
    else:
        grad_thresh=(10,255)
        mag_thresh=(10,255)
        l_thresh=(125,255)
        g_thresh=(120,255)

    gradx = abs_sobel_thresh(img, orient='x', sobel_kernel=kernel, thresh=grad_thresh)
    grady = abs_sobel_thresh(img, orient='y', sobel_kernel=kernel, thresh=grad_thresh)
    mag_binary = mag_threshold(img, sobel_kernel=kernel, mag_thresh=mag_thresh)
    
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    lbinary = binary_thresh(L, l_thresh)
    
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    gbinary = binary_thresh(R, g_thresh)
    
    combined = np.zeros_like(gradx)
    combined[(grady == 1) | (mag_binary == 1) | (lbinary ==1 ) | (gbinary == 1)] = 1
    
    return combined

def getPerspectiveTransform(src, dst, img):
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (img.shape[0], img.shape[1]), flags=cv2.INTER_LINEAR)
    return warped

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    
    # Set the width of the windows +/- margin
    margin = 80
    
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 5) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 5) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
#     print(np.poly1d(left_fit))

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = np.polyval(left_fit, ploty)
        right_fitx = np.polyval(right_fit, ploty)
#         left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
#         right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [0, 0, 255]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
    
    return left_fitx, right_fitx, ploty, out_img

def yoloToNormal(yolo_x, yolo_y, yolo_width, yolo_height):
    
    image_width, image_height = 1920, 1088
    x_center = yolo_x * image_width
    y_center = yolo_y * image_height
    width = yolo_width * image_width
    height = yolo_height * image_height

    x_left = (yolo_x - yolo_width/2) * image_width
    y_top = (yolo_y - yolo_height/2) * image_height

    points = [(x_left+width, y_top), (x_left, y_top),(x_left, y_top+height), (x_left+width, y_top+height)]
    return points

def getFillPoly(warped, i_for_txt):    
    LXY = [i for i in zip(left_fitx, ploty)]
    RXY = [i for i in zip(right_fitx, ploty)]
    
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    arr = np.array( [LXY + RXY[::-1]], dtype=np.int32 )

    overlay = warped.copy()
    
# car obstacles
###################################################################################################################################################################################
    caution = ''
    bin_img = np.zeros_like(warped[:,:,0])
    bin_filled = cv2.fillPoly(bin_img, arr, 1)
    bin_unwarped = getPerspectiveTransform(dst, src, bin_filled)
    bin_obstacles = bin_unwarped.copy()
    try:
      # txt = open('/content/yolov5/runs/detect/exp/labels/Day2_1.txt',"r").read()
      txt = open('/content/yolov5/runs/detect/'+exp+'/labels/' + video[:-4] + '_'+ str(i_for_txt) +'.txt',"r").read()
      
      for txt in txt.split('\n')[:-1]: #loop the value for each line
          # text to float
          yolo_x, yolo_y, yolo_width, yolo_height = float(txt.split()[1]), float(txt.split()[2]), float(txt.split()[3]), float(txt.split()[4])
          
          points = np.array(yoloToNormal(yolo_x, yolo_y, yolo_width, yolo_height)).astype('int')
          # draw line to test obstacles
          bin_obstacles = cv2.polylines(bin_obstacles.copy(),[points],True,0,5)
          
  #     print(sum(sum(bin_obstacles)), sum(sum(bin_unwarped)))
      if(sum(sum(bin_obstacles))!=sum(sum(bin_unwarped))):
          cv2.fillPoly( overlay, arr, (0,0,255,125))
          caution = 'Too close!'
      else:
          cv2.fillPoly( overlay, arr, (255,255,0,125))
          caution = 'Safe!'
    except:
      print('No label!')
      cv2.fillPoly( overlay, arr, (255,255,0,125))
      caution = 'Safe!'

####################################################################################################################################################################################
    alpha = 0.4
    image_new = cv2.addWeighted(overlay, alpha, warped, 1 - alpha, 0)
    return image_new,caution

def combineMask(img_ori,unwarped):
    #create a inverse mask
    img2gray = cv2.cvtColor(unwarped,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # black-out the area of img1
    img1_bg = cv2.bitwise_and(img_ori,img_ori,mask = mask_inv)
    img2_fg = unwarped

    # Put logo in ROI and modify the main image
    final = cv2.add(img1_bg, img2_fg)
    return final

def combine_sw_final(final, out_img, caution, i_for_txt):
    image = np.zeros_like(out_img.copy())
    LXY = [i for i in zip(left_fitx, ploty)]
    RXY = [i for i in zip(right_fitx, ploty)]

    ptsl = np.array(LXY,np.int32)
    ptsl = ptsl.reshape((-1, 1, 2)) 
    ptsr = np.array((RXY),np.int32)
    ptsr = ptsr.reshape((-1, 1, 2)) 
    isClosed = False

    # Blue color in BGR 
    color = (255, 0, 0) 

    # Line thickness of 2 px 
    thickness = 10

    image = cv2.polylines(image, [ptsl], isClosed, color, thickness) 
    image = cv2.polylines(image, [ptsr], isClosed, color, thickness) 
    sliding_window = cv2.add(image,out_img)
    
    widht = sliding_window.shape[0]//3
    height = sliding_window.shape[1]//3
    resized_sliding_window = cv2.resize(sliding_window, (widht,height),interpolation = cv2.INTER_AREA)

    image_border = final.copy()
    image_border[:resized_sliding_window.shape[0], :resized_sliding_window.shape[1]] = resized_sliding_window

    # put text 
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (700, 100) 
    fontScale = 3
    thickness = 10

    if caution == 'Safe!':
        image_border = cv2.putText(image_border, 'Caution: ' + caution, org, font,  
                       fontScale, (0, 180, 0) , thickness, cv2.LINE_AA)
    else:
        image_border = cv2.putText(image_border, 'Caution: ' + caution, org, font,  
                       fontScale, (0, 0, 255) , thickness, cv2.LINE_AA)
            
    image_border = cv2.putText(image_border, 'Frame: ' + str(i_for_txt), (700, 200) , font,  
                       fontScale, (180, 0, 0) , thickness, cv2.LINE_AA)   
    return image_border


def videoToFrame(cap):
    frames = []
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            frames.append(frame)
        else:
            break
    return frames

In [ ]:
cap = cv2.VideoCapture('/content/gdrive/My Drive/WongManKiat_1161204289/object_detection_video/' + video)

#xiaomi mai
# src = np.float32([[400,950], [900, 520], [1030, 520], [1650,950]]) #high way
src = np.float32([[ 350.,  950.], [ 800.,  600.], [1080.,  600.], [1400.,  950.]]) #normal road

src = np.float32([[400,950], [850, 600], [1100, 600], [1440,950]]) #mmu petronus road
dst = np.float32([[300,1750], [300,100], [900,100], [900,1750]]) 


# #xiaomi night
# src = np.float32([[80,950], [870, 520], [1030, 520], [1430,950]])
# dst = np.float32([[300,1750], [300,100], [900,100], [900,1750]])

i_for_txt = 1
loop_start = time.time()

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
vid_writer = cv2.VideoWriter('/content/gdrive/My Drive/WongManKiat_1161204289/lane_detection_video/'+video, cv2.VideoWriter_fourcc(*'XVID'), fps, (w, h), True)

start = time.time()
while(cap.isOpened()):
    ret, f = cap.read()
    if ret == True:

        start = time.time()      
        try:
          thd = combined_threshold(f)
          warped_thd = getPerspectiveTransform(src, dst, thd)
          binary_warped = warped_thd.copy().astype(np.uint8) 
          left_fitx, right_fitx, ploty, out_img = fit_polynomial(binary_warped)
          warped = getPerspectiveTransform(src, dst, f)
          warped_filled, caution = getFillPoly(warped, i_for_txt)
          unwarped = getPerspectiveTransform(dst, src, warped_filled)
          final = combineMask(f,unwarped)
          combined = combine_sw_final(final, out_img, caution, i_for_txt-1)
          vid_writer.write(combined)
          # break
        except TypeError:
          print('error main')
          vid_writer.write(combined)
        
        end = time.time()
        print('Image {}/{} - {:.3f} s'.format(i_for_txt, length, end - start))
        i_for_txt += 1

    else:
        break
loop_end = time.time()
print('Done {:.3f} s'.format(loop_end - loop_start))    
vid_writer.release()

Image 1/227 - 0.454 s
Image 2/227 - 0.358 s
No label!
Image 3/227 - 0.354 s
No label!
Image 4/227 - 0.348 s
No label!
Image 5/227 - 0.354 s
Image 6/227 - 0.349 s
Image 7/227 - 0.363 s
Image 8/227 - 0.368 s
Image 9/227 - 0.414 s
No label!
Image 10/227 - 0.383 s
No label!
Image 11/227 - 0.382 s
No label!
Image 12/227 - 0.383 s
No label!
Image 13/227 - 0.364 s
No label!
Image 14/227 - 0.371 s
No label!
Image 15/227 - 0.361 s
No label!
Image 16/227 - 0.369 s
No label!
Image 17/227 - 0.368 s
No label!
Image 18/227 - 0.360 s
No label!
Image 19/227 - 0.375 s
No label!
Image 20/227 - 0.365 s
No label!
Image 21/227 - 0.352 s
No label!
Image 22/227 - 0.365 s
No label!
Image 23/227 - 0.364 s
No label!
Image 24/227 - 0.356 s
No label!
Image 25/227 - 0.364 s
No label!
Image 26/227 - 0.363 s
Image 27/227 - 0.372 s
No label!
Image 28/227 - 0.370 s
No label!
Image 29/227 - 0.370 s
No label!
Image 30/227 - 0.360 s
Image 31/227 - 0.369 s
Image 32/227 - 0.375 s
No label!
Image 33/227 - 0.379 s
No label!
